<a href="https://colab.research.google.com/github/Ashis-Palai/Cancer_Information_RAG_GenAI/blob/main/GenAI_Cancer_Information_Powered_by_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -O "https://raw.githubusercontent.com/Ashis-Palai/Cancer_Information_RAG_GenAI/main/requirements.txt"

# **DATA EXTRACTION:**



## **Sources**

* **Cervical , Breast & Oral Cancers:**


> ```
https://tmc.gov.in/ncg/docs/PDF/DraftGuidelines/Preventive/3_%20NCG_INDIA_Rev_Preventive%20Oncology_Primary_Care.pdf
```


* **Ovarian_Cancer:**

> ```
https://main.icmr.nic.in/sites/default/files/guidelines/Ovarian_Cancer.pdf
```


* **Generic Information About Cancer**

>```
https://www.mayoclinic.org/diseases-conditions/cancer/diagnosis-treatment/drc-20370594
```



* **Breast Cancer:**

>```
https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html
```






In [3]:
!curl -O "https://raw.githubusercontent.com/Ashis-Palai/Cancer_Information_RAG_GenAI/main/helper.py"
!curl -O "https://raw.githubusercontent.com/Ashis-Palai/Cancer_Information_RAG_GenAI/main/requirements.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3168  100  3168    0     0  14661      0 --:--:-- --:--:-- --:--:-- 14734
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   155  100   155    0     0   1351      0 --:--:-- --:--:-- --:--:--  1359


In [4]:
all_url = [ 'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/american-cancer-society-recommendations-for-the-early-detection-of-breast-cancer.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/mammograms.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-ultrasound.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-mri-scans.html',
'https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-cancer-signs-and-symptoms.html',
'https://tmc.gov.in/ncg/docs/PDF/DraftGuidelines/Preventive/3_%20NCG_INDIA_Rev_Preventive%20Oncology_Primary_Care.pdf',
'https://main.icmr.nic.in/sites/default/files/guidelines/Ovarian_Cancer.pdf'
            ]

In [5]:
from helper import extract_headings_and_content , word_wrap
for i, url in enumerate(all_url, start=1):
    result_message = extract_headings_and_content(url, i)
    print(result_message)

Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection.html. File saved as data/html_data/html_file_1.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/american-cancer-society-recommendations-for-the-early-detection-of-breast-cancer.html. File saved as data/html_data/html_file_2.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/mammograms.html. File saved as data/html_data/html_file_3.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-ultrasound.html. File saved as data/html_data/html_file_4.html
Extraction and saving successful for https://www.cancer.org/cancer/types/breast-cancer/screening-tests-and-early-detection/breast-mri-scans.html. File saved as data/html_data/html_file_5.htm

# **DATA TRANSFORMATION USING LANGCHAIN**

In [6]:
from langchain_community.document_loaders import PyPDFLoader , UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter , SentenceTransformersTokenTextSplitter
import glob

In [7]:


base_path = '/content/data'

# List all HTML files
html_files = glob.glob(f"{base_path}/html*/*.html")

# List all PDF files
pdf_files = glob.glob(f"{base_path}/pdf*/*.pdf")


all_results = [PyPDFLoader(file).load() if file.endswith(".pdf") else UnstructuredHTMLLoader(file).load() for file in pdf_files + html_files]



In [8]:
t = PyPDFLoader('/content/data/pdf_data/pdf_file_7.pdf').load()

In [9]:
t[0].metadata

{'source': '/content/data/pdf_data/pdf_file_7.pdf', 'page': 0}

In [10]:
h = UnstructuredHTMLLoader('/content/data/html_data/html_file_1.html').load()

In [11]:
h[0].metadata

{'source': '/content/data/html_data/html_file_1.html'}

In [12]:
total_pages = sum([len(i) for i in all_results ])
total_pages

107

In [13]:
all_text_data = [j.page_content.strip() for i in all_results for j  in i]

In [14]:
len(all_text_data)

107

In [15]:
all_text_data = [ i for i in all_text_data if i] # To exclude the blank pages

In [16]:
len(all_text_data)

102

In [17]:
splitter_1 =  RecursiveCharacterTextSplitter(
    separators= ['\n\n\n\n\n','\n\n\n\n','\n\n\n','\n\n','\n','.',',',' '],
    chunk_size = 1000 ,chunk_overlap = 100)

splitter_2 = SentenceTransformersTokenTextSplitter(tokens_per_chunk=256,chunk_overlap=10)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
prefinal_data = splitter_1.split_text('\n'.join(all_text_data))
len(prefinal_data)

314

In [19]:
final_data = []
[final_data.extend(splitter_2.split_text(text)) for text in prefinal_data]

len(final_data)

386

In [20]:
prefinal_data[0]

'Coordinated by:\nDivision of Non Communicable Diseases\nIndian Council of Medical Research  \nAnsari Nagar, New Delhi – 110029\n2019CONSENSUS DOCUMENT FOR\nMANAGEMENT OF\nEPITHELIAL OVARIAN CANCER\nPrepared as an outcome of ICMR Subcommittee on\nEpithelial Ovarian Cancer\nProf. Balram Bhargava  \nSecretary,  \nDepartment of Health Research  \nand Director General, ICMR\nPublished in 2019\nHead (Publication & Information) : Dr. Neeraj Tandon\nCompiled & Edited by: Dr. Hemant Tongaonkar & Dr. Tanvir Kaur\nProduction Controller : Mr. JN Mathur \nPublished by the Division of Publication and Information on behalf of the Secretary DHR & DG, ICMR,  \nNew Delhi. \nDesigned & Printed at M/s Royal Offset Printers, A-89/1, Naraina Industrial Area, Phase-I, New Delhi-110028  \nMobile: 9811622258Disclaimer\nThis consensus document represents the current thinking of experts on the topic based on available \nevidence. This has been developed by national experts in the field and does not in any way b

In [21]:
final_data[0]

'coordinated by : division of non communicable diseases indian council of medical research ansari nagar, new delhi – 110029 2019consensus document for management of epithelial ovarian cancer prepared as an outcome of icmr subcommittee on epithelial ovarian cancer prof. balram bhargava secretary, department of health research and director general, icmr published in 2019 head ( publication & information ) : dr. neeraj tandon compiled & edited by : dr. hemant tongaonkar & dr. tanvir kaur production controller : mr. jn mathur published by the division of publication and information on behalf of the secretary dhr & dg, icmr, new delhi. designed & printed at m / s royal offset printers, a - 89 / 1, naraina industrial area, phase - i, new delhi - 110028 mobile : 9811622258disclaimer this consensus document represents the current thinking of experts on the topic based on available evidence. this has been developed by national experts in the field and does not in any way bind'

# **DATA LOADING**

## **Embedding : GooglePalmEmbeddings**

In [22]:
from langchain_community.embeddings import GooglePalmEmbeddings
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
emb = GooglePalmEmbeddings(model_name='models/embedding-gecko-001',google_api_key=GOOGLE_API_KEY)
print(f"Out put dimension: {len(emb.embed_query('Hi How are you?'))}")

Out put dimension: 768


## **Data Base : Pinecone Vector DB**

In [ ]:
from pinecone import Pinecone, ServerlessSpec, PodSpec


p = Pinecone(api_key= PINECONE_API_KEY)

In [ ]:
# p.create_index('cancer-retrieval',
#               dimension=768,
#               metric='cosine',
#               spec = PodSpec(environment='gcp-starter',
#                              pod_type='starter',
#                              pods = 1))
# index =  p.Index('cancer-retrieval')

In [ ]:
index =  p.Index('cancer-retrieval')

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
total_vector = index.describe_index_stats()['total_vector_count']

In [ ]:
(final_data[200])

'survival among the study participants in both groups ( p =. 0001 ). in subset analysis, patients with stage iiic disease and metastatic tumour < 45 mm had better survival than those with stage iv disease or larger > 45 mm tumours. other significant prognostic factors included : histological type ( p =. 0003 ), largest tumor size at randomization ( p =. 0008 ) and disease stage ( iiic vs. iv ) ( p =. 0008 ). the results of this study provide support for the use of nact / interval surgery in advanced ovarian cancer, but do not detract from the need for optimal surgery since this was still the strongest predictor of outcome. many critics of the eortc study have cited better optimal reduction rates and pfs / os with primary debulking surgery ( pds ) from single institution american series, compared to the pds arm of eortc study, thereby questioning the quality of surgery in this trial ( 5 ) however, this trial only accrued higher risk'

In [ ]:
embeddings = emb.embed_documents(final_data)

In [ ]:
final_db_data =[]
for i,j in enumerate(final_data):
  final_db_data.append({'id':str(total_vector + i),'values':embeddings[i],'metadata':{'text':j}})

In [ ]:
index.upsert(final_db_data)

{'upserted_count': 385}

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00385,
 'namespaces': {'': {'vector_count': 385}},
 'total_vector_count': 385}

# **DATA VERIFICATION AFTER LOADING**

In [ ]:
from langchain_community.embeddings import GooglePalmEmbeddings
from google.colab import userdata
from pinecone import Pinecone

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

p = Pinecone(api_key=PINECONE_API_KEY)
index = p.Index('cancer-retrieval')

In [ ]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00385,
 'namespaces': {'': {'vector_count': 385}},
 'total_vector_count': 385}

In [ ]:
query = ' Oral Cancer?'
embed = emb.embed_query(query)
res = index.query(vector=embed, top_k=15, include_metadata=True)
text = [r['metadata']['text'] for r in res['matches']]
print(word_wrap('\n'.join(text)))



# **LLM IMPLEMENTATION (GOOGLE GEMINI PRO)**

In [ ]:
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI

temperature = 0.1
# max_tokens = 500
# Create a ChatGoogleGenerativeAI instance for Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key= GOOGLE_API_KEY, temperature=temperature
)


## **Prompt Construction**

In [ ]:
query = ' Oral Cancer?'
embed = emb.embed_query(query)
res = index.query(vector=embed, top_k=15, include_metadata=True)
text = [r['metadata']['text'] for r in res['matches']]
context = '\n'.join(text)
prompt = f'''You are an expert health care CANCER information provider specifically  Cervical , Breast, Oral ,Ovarian related Cancers.
You will also be given CONTEXT based on this context you have to answer or give brief summary about the question.
Make sure you should not give false or Hallucinate or give answer randomly , answer should be combination of your true knowledge and context given.
question Is:{query}
Context Is:{context}
'''


prompt = prompt



# Send the prompt to Gemini and get the response
response = llm.invoke( prompt)



In [ ]:
response

AIMessage(content='question: What is Oral Cancer?\nAnswer: Oral cancer is an important public health problem with increasing incidence and late - stage presentation.\nContext: Oral cancer is an important public health problem with increasing incidence and late - stage prese ntation. cancers of the mouth, pharynx and larynx taken together are the seventh most commonly occurring type of cancer worldwide.')

In [ ]:
print(word_wrap(response.content))

question: Oral Cancer?
Answer: Oral cancer is an important public
health problem with increasing incidence and late-stage presentation.
Cancers of the mouth, pharynx, and larynx taken together are the
seventh most commonly occurring type of cancer worldwide. India
reported lip, oral cavity cancer at ASIR of 9.1 for both sexes combined
and 13.9 for men. Oral cancer thus accounts as the highest incidence of
malignancy in males and the second highest in females in India.
Smoking, alcohol use, smokeless tobacco use, and HPV infection are the
major risk factors for oral cavity cancer.
Context: and neck cancer
epidemiology consortium. cancer epidem iol biomar kers prev. 20 09 ; 18
: 541 - 550. 3. wen cp, tsai mk, chung ws, et al. cancer risks from
betel quid chewing beyond oral cancer : a multiple - site carcinogen
when acting with smo king. cancer causes control. 2010 ; 21 : 1427 -
1435. 4. kreimer ar, clifford gm, boyle p, franceschi s. human
papillomavi rus types in head and neck squamous

# **LANGCHAIN EXPRESSION LANGUAGE**

In [27]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import GooglePalmEmbeddings
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')


In [43]:
llm = ChatGoogleGenerativeAI(model='gemini-pro',google_api_key=GOOGLE_API_KEY,convert_system_message_to_human=True)
emb = GooglePalmEmbeddings(google_api_key=GOOGLE_API_KEY)
retriever = PineconeVectorStore(index_name='cancer-retrieval',embedding=emb,pinecone_api_key=PINECONE_API_KEY).as_retriever()

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = '''You are an expert health care CANCER information provider specifically  Cervical , Breast, Oral ,Ovarian related Cancers.
You will also be given CONTEXT based on this context you have to answer or give brief summary about the question.
Make sure you should not give false or Hallucinate or give answer randomly , answer should be combination of your true knowledge and context given.
question Is:{query}
Context Is:{context}
answer:'''

In [ ]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | PromptTemplate.from_template(prompt)
    | llm
    | StrOutputParser()
)


In [ ]:
chain.invoke('What is Oral Cancer?')

'Oral cancer is a type of cancer that develops in the oral cavity, which includes the lips, tongue, cheeks, gums, and floor of the mouth. It is the sixth most common type of cancer worldwide, and is more common in men than in women. The main risk factors for oral cancer are tobacco use, alcohol consumption, and human papillomavirus (HPV) infection. Oral cancer can be treated with surgery, radiation therapy, chemotherapy, and targeted therapy.'

In [ ]:
llm.invoke('What is Oral Cancer?')

AIMessage(content='**Oral Cancer**\n\nOral cancer is a malignant disease that develops in the tissues of the mouth and oropharynx (the area at the back of the mouth, including the tonsils and base of the tongue).\n\n**Types of Oral Cancer:**\n\n* **Squamous cell carcinoma:** The most common type of oral cancer, accounting for over 90% of cases.\n* **Adenocarcinoma:** Cancer that develops in the salivary glands.\n* **Melanoma:** Cancer that develops in the pigment-producing cells of the mouth.\n* **Sarcoma:** Cancer that develops in the connective tissues of the mouth.\n\n**Risk Factors:**\n\n* **Tobacco use:** Smoking, chewing tobacco, and smokeless tobacco increase the risk significantly.\n* **Alcohol consumption:** Excessive alcohol intake can irritate the lining of the mouth and increase the risk of cancer.\n* **Human papillomavirus (HPV) infection:** Certain strains of HPV, particularly HPV-16 and HPV-18, are associated with an increased risk of oral cancer.\n* **Poor oral hygiene:

# **RESULT EVALUATION**

## **Using Langchain QAGenerateChain**

> **have to pass the whole data to LLM to generate question and answer, llm use is maximum with this process , aditional token cost**

In [62]:
from langchain.evaluation.qa import QAGenerateChain
from langchain.chains import RetrievalQA
example_gen_chain = QAGenerateChain.from_llm(llm)

In [75]:

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in final_data[:10]]
)


In [73]:
new_examples = [i['qa_pairs'] for i in new_examples]
new_examples[:5]

In [47]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [48]:
qa_refine = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    verbose=True
)

In [49]:
predictions_stuff = qa_stuff.batch(new_examples)
predictions_refine = qa_refine.batch(new_examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [50]:
from langchain.evaluation.qa import QAEvalChain
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs_stuff = eval_chain.evaluate(new_examples, predictions_stuff)
graded_outputs_refine = eval_chain.evaluate(new_examples, predictions_refine)

In [53]:
for i, eg in enumerate(new_examples):
    print(f"Example {i}:")
    print("Question: " + predictions_stuff[i]['query'])
    print("Real Answer: " + predictions_stuff[i]['answer'])
    print("Predicted Answer: " + predictions_stuff[i]['result'])
    print("Predicted Grade: " + graded_outputs_stuff[i]['results'])
    print('*'*20)
    if i == 5:
      break


Example 0:
Question: Who is the head of the Publication & Information department responsible for this document?
Real Answer: Dr. Neeraj Tandon
Predicted Answer: I cannot find the answer to your question in the context you provided.
Predicted Grade: CORRECT
********************
Example 1:
Question: What is the purpose of the consensus document for the management of epithelial ovarian cancer?
Real Answer: The consensus document aims to provide clear, consistent, succinct, and evidence-based guidance for clinicians in managing epithelial ovarian cancer. It serves as a reference for healthcare professionals to make informed decisions regarding the diagnosis, treatment, and follow-up care of patients with this type of cancer.
Predicted Answer: The purpose of the consensus document for the management of epithelial ovarian cancer is to provide clear, consistent, succinct, evidence-based guidance for the management of epithelial ovarian cancer.
Predicted Grade: CORRECT
********************
Exa

In [ ]:
for i, eg in enumerate(new_examples):
    print(f"Example {i}:")
    print("Question: " + predictions_refine[i]['query'])
    print("Real Answer: " + predictions_refine[i]['answer'])
    print("Predicted Answer: " + predictions_refine[i]['result'])
    print("Predicted Grade: " + graded_outputs_refine[i]['results'])
    print('*'*20)
    if i == 5:
      break


In [54]:
total_examples = len(new_examples)
correct_predictions_stuff = 0
correct_predictions_refine = 0

for i, eg in enumerate(new_examples):
    # Assuming 'CORRECT' indicates correct prediction in the graded output
    actual_label_stuff = graded_outputs_stuff[i]['results'] == 'CORRECT'
    actual_label_refine = graded_outputs_refine[i]['results'] == 'CORRECT'



    if actual_label_stuff:
        correct_predictions_stuff += 1
    if actual_label_refine:
        correct_predictions_refine += 1

accuracy_stuff = correct_predictions_stuff / total_examples * 100 if total_examples > 0 else 0
accuracy_refine = correct_predictions_refine / total_examples * 100 if total_examples > 0 else 0


print(f'Accuracy_stuff_method: {accuracy_stuff}% & Accuracy_refine_method: {accuracy_refine}%')


Accuracy: 80.0%


## **Using RAGAS**